# DoNotMakeMyTrip – AI Airline Assistant with Live Flight Prices, Voice & Visuals
This project builds an interactive airline assistant for a fictional company called **DoNotMakeMyTrip**. The assistant can:

- Answer questions about flight prices using live data from the **Amadeus** sandbox API.
- Ask clarifying questions (origin, destination, dates) before searching.
- “Book” flights and generate a **fake booking confirmation** plus a **boarding‑pass style image**.
- Generate a **company logo** and **destination images** using the OpenAI Images API.
- Speak all responses using **OpenAI Text‑to‑Speech** (TTS).
- Provide a full chat experience via a **Gradio** web app with:
  - Chat window
  - Trip image panel (logo / destination / boarding pass)
  - Audio playback panel

## 1. Imports and Model Initialization

In [1]:
import os
import json
import random
import string
import base64
import io
import gradio as gr
import pyaudio
import requests
from datetime import datetime, date
from IPython.display import Audio, display
from io import BytesIO
from PIL import Image
import tempfile
from dotenv import load_dotenv
from openai import OpenAI
from google.genai import types

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
MODEL = 'gpt-4.1-mini'
MODEL_IMAGE = 'dall-e-3'
MODEL_SPEECH = 'gpt-4o-mini-tts'

In [4]:
try:
    openai = OpenAI(api_key=openai_api_key)
    print("OpenAI Client initialized successfully!")
except Exception as e:
    print(f"Error initializing OpenAI Client: {e}")
    print("Ensure your OPENAI_API_KEY is correctly set as an environment variable.")
    exit() 

OpenAI Client initialized successfully!


## 2. Company Logo Generation (`generate_company_logo`)
### How it works

- Calls `openai.images.generate` with:
  - `model=MODEL_IMAGE` (`dall-e-3`)
  - `size="1024x1024"`
  - `n=1` (one image)
  - `quality="hd"`
  - `response_format="b64_json"` so the API returns base64‑encoded image data.
- Checks `response.data` and extracts `b64_json` from the first item.
- Decodes base64 to bytes and opens it as a `PIL.Image`.
- Returns the `Image` object.

In [5]:
def generate_company_logo(company_name: str = "DoNotMakeMyTrip",):
    prompt = (
    f"Create a clean, modern, professional logo for an airline and travel company called '{company_name}'. "
    "The logo must clearly display the full company name as the main text, in a bold, easily readable font. "
    "Combine the text with a simple, memorable icon that hints at air travel or journeys "
    "(for example a stylized airplane, wing, flight path, or location pin). "
    "Use a flat vector style with a limited color palette (such as teal and dark blue or other cool tones), "
    "with strong contrast so the name stands out clearly. "
    "The design should work well in a horizontal layout for a website header and also be adaptable to an app icon. "
    "Keep the overall look minimal and uncluttered: white or very light background, no photo-realistic elements, "
    "no gradients, and no tiny or unreadable text."
)
    
    response = openai.images.generate(
        model=MODEL_IMAGE,
        prompt=prompt,
        size="1024x1024",   # adjust as needed: "512x512", "1792x1024", etc.
        n=1,
        quality="hd",    
        response_format="b64_json",   # or "standard"
    )

    # Take the first generated image
    if not response.data:
        raise ValueError("No image returned by OpenAI.")

    image_b64 = response.data[0].b64_json
    image_bytes = base64.b64decode(image_b64)
    return Image.open(BytesIO(image_bytes))

## 3. Destination Image Generation (`fetch_image`)
### How it works

- Constructs a rich prompt including the `city` name in multiple places.
- Calls `openai.images.generate` (same setup as the logo function).
- Decodes the base64 image data into a `PIL.Image`.
- Returns the image.

In [6]:
def fetch_image(city):
    prompt = (
    f"A high-quality, photo-realistic travel photograph set in {city}, "
    f"featuring its most recognizable landmarks and everyday street scenes. "
    f"Show real, diverse people casually enjoying the city — walking, talking, eating local food, or sightseeing — "
    f"with natural, unscripted expressions and body language. "
    f"Include authentic architectural details, signage, and local textures that clearly evoke {city}, "
    f"with natural lighting and realistic weather for the location and time of day. "
    f"The composition should feel like a candid shot from a professional travel photographer, "
    f"using a full-frame DSLR, shallow depth of field, and subtle color grading, "
    f"resulting in an immersive, warm, and believable scene that looks like a real photograph, not digital art."
)

    response = openai.images.generate(
        model=MODEL_IMAGE,
        prompt=prompt,
        size="1024x1024",   # adjust as needed: "512x512", "1792x1024", etc.
        n=1,
        quality="hd",    
        response_format="b64_json",   # or "standard"
    )

    # Take the first generated image
    if not response.data:
        raise ValueError("No image returned by OpenAI.")

    image_b64 = response.data[0].b64_json
    image_bytes = base64.b64decode(image_b64)
    return Image.open(BytesIO(image_bytes))

## 4. Text‑to‑Speech (`talk`)

### What it does

`talk(message: str) -> str | None`:

- Sends the assistant’s reply text to OpenAI’s TTS model.
- Saves the resulting MP3 audio to a temporary file.
- Returns the **file path** to that audio file.

### How it works

- Calls `openai.audio.speech.create` with:
  - `model=MODEL_SPEECH` (`gpt-4o-mini-tts`)
  - `voice="onyx"`
  - `input=message`
- Extracts `response.content` as raw MP3 bytes.
- Writes the bytes to a uniquely named temp file using `tempfile.NamedTemporaryFile(delete=False)`.
- Returns the temp file path.

In [7]:
def talk(message: str) -> str | None:
    """
    Generate TTS audio for the message and return a temporary MP3 file path.
    The file path is used by Gradio's Audio component (autoplay in UI).
    """
    try:
        response = openai.audio.speech.create(
            model=MODEL_SPEECH,
            voice="onyx",
            input=message,
        )
    except Exception as e:
        print("[Speech Error] TTS call failed:", e)
        return None

    audio_bytes = response.content  # MP3 bytes

    # Create a unique temp file so the browser doesn't cache the same filename
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
        f.write(audio_bytes)
        temp_path = f.name

    return temp_path

## 5. Amadeus Setup and Authentication (`BASE_URL`, credentials, `get_amadeus_token`)

### What it does

- `BASE_URL` is set to the **Amadeus test environment**: `https://test.api.amadeus.com`.

In [8]:
BASE_URL = "https://test.api.amadeus.com"  

In [9]:
amadeus_client_id = os.getenv("AMADEUS_CLIENT_ID")
amadeus_client_secret = os.getenv("AMADEUS_CLIENT_SECRET")

if amadeus_client_id:
    print(f"Amadeus CLIENT_ID exists and begins: {amadeus_client_id[:6]}...")
else:
    print("AMADEUS_CLIENT_ID not set")

if amadeus_client_secret:
    print(f"Amadeus CLIENT_SECRET is set (length: {len(amadeus_client_secret)} characters)")
else:
    print("AMADEUS_CLIENT_SECRET not set")

if not (amadeus_client_id and amadeus_client_secret):
    raise RuntimeError(
        "Missing Amadeus credentials. Please set AMADEUS_CLIENT_ID and "
        "AMADEUS_CLIENT_SECRET in your .env file."
    )

Amadeus CLIENT_ID exists and begins: 4zce1w...
Amadeus CLIENT_SECRET is set (length: 16 characters)


### `get_amadeus_token()`

This function implements the **OAuth2 client_credentials** flow:

- URL: `BASE_URL + /v1/security/oauth2/token`.
- Sends a POST request with:
  - `grant_type=client_credentials`
  - `client_id=AMADEUS_CLIENT_ID`
  - `client_secret=AMADEUS_CLIENT_SECRET`
- On success, returns `access_token`.
- On error, prints debug information and returns `None`.


In [10]:
def get_amadeus_token() -> str | None:
    """
     In this function, it:
     * Sends your AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET to Amadeus OAuth2 endpoint.
     * If the credentials are valid, Amadeus returns an access token, as a string.
     * With that token, you then connect to the actual Amadeus APIs (flight offers, airlines, locations, etc.).
    """
    url = f"{BASE_URL}/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": amadeus_client_id,
        "client_secret": amadeus_client_secret,
    }

    try:
        response = requests.post(url, headers=headers, data=data, timeout=10)
        response.raise_for_status()
        return response.json().get("access_token")
    except requests.exceptions.HTTPError:
        print("[Amadeus Auth HTTP error]")
        print("Status:", response.status_code)
        print("Body:", response.text)
    except requests.exceptions.RequestException as e:
        print("[Amadeus Auth network error]", e)

    return None

## 6. Airline Name Lookup (`get_airline_name`)

### What it does

`get_airline_name(code: str, token: str) -> str`:

- Takes an IATA airline code (e.g. `"EK"`) and an access token.
- Calls the Amadeus airlines reference endpoint to retrieve the **business name** (e.g. `"Emirates"`).
- Returns the business name or falls back to the code itself if anything fails.

### How it works

- URL: `BASE_URL + /v1/reference-data/airlines`.
- Headers: `Authorization: Bearer <token>`.
- Params: `{"airlineCodes": code}`.
- Performs a GET request, checks `response.raise_for_status()`, parses JSON.
- If `data["data"]` exists and non‑empty, returns `data["data"][0]["businessName"]`.
- On request errors, prints an error and returns `code`.

In [11]:
def get_airline_name(code: str, token: str) -> str:
    """
    Takes that code ("EK") and your access token.
    Calls the Amadeus “reference data / airlines” endpoint.
    Returns the airlines business name.
    If anything goes wrong, it falls back to returning the code itself.
    """
    url = f"{BASE_URL}/v1/reference-data/airlines" # Builds the URL for Amadeus’ Airlines reference data
    headers = {"Authorization": f"Bearer {token}"} # Sets the Authorization header with your bearer token
    params = {"airlineCodes": code} # sends codes like "EK"

    try:
        response = requests.get(url, # URL: reference-data/airlines
        headers=headers, # Headers: includes Authorization
        params=params, # Params: airlineCodes
        timeout=10)
        response.raise_for_status() # raise_for_status() throws an error if status is not 2xx.
        # If the status code is 2xx (success) → do nothing, continue.
        # If the status code is 4xx (client error, e.g. 400, 401, 404) or 5xx (server error, e.g. 500) → raise an exception (HTTPError).
        data = response.json() # If successful, parses the JSON body into data.
    except requests.exceptions.RequestException as e: # Catches any network or HTTP error (e.g. bad token, timeouts).
        print(f"[ERROR] Airline lookup failed for {code}: {e}")
        return code

    if "data" in data and data["data"]: # Checks that the JSON contains a non‑empty "data" array.
        return data["data"][0].get("businessName", code) # Returns the "businessName" field if it exists, otherwise returns code.

    return code # Else, Returns code as a fallback.

#  ----------- EXAMPLE ----------
# So, in the best case:
#     Input: "EK" and a valid token
#     Output: "Emirates"
# If something fails:
#     Output: still "EK" (so your broader logic does not crash).

## 7. City Name to IATA Code (`COMMON_CITY_CODES`, `city_code_cache`, `get_city_code`)

### `COMMON_CITY_CODES`

- A hard‑coded dictionary mapping common **Indian cities** to their IATA codes, for example:
  - `"delhi": "DEL"`
  - `"mumbai": "BOM"`
  - `"chennai": "MAA"`
- Used as a quick, local lookup to avoid API calls and speed up common queries.

In [12]:
COMMON_CITY_CODES = {
    "delhi": "DEL",
    "mumbai": "BOM",
    "chennai": "MAA",
    "kolkata": "CCU",
    "bengaluru": "BLR",
    "hyderabad": "HYD",
    "patna": "PAT",
    "raipur": "RPR",
    "panaji": "GOI",
    "chandigarh": "IXC",
    "srinagar": "SXR",
    "ranchi": "IXR",
    "thiruvananthapuram": "TRV",
    "bhopal": "BHO",
    "imphal": "IMF",
    "aizawl": "AJL",
    "bhubaneswar": "BBI",
    "jaipur": "JAI",
    "agartala": "IXA",
    "lucknow": "LKO",
    "dehradun": "DED",

    # Union territories
    "port blair": "IXZ",
    "leh": "IXL",
    "puducherry": "PNY",

    # Other major cities
    "ahmedabad": "AMD",
    "surat": "STV",
    "coimbatore": "CJB",
    "vizag": "VTZ",
    "vijayawada": "VGA",
    "nagpur": "NAG",
    "indore": "IDR",
    "kanpur": "KNU",
    "varanasi": "VNS",
}

### `city_code_cache`

- A simple in‑memory cache:
  - Key: normalized city name (lowercase).
  - Value: IATA code.
- Prevents repeated lookups for the same city in one session.

### `get_city_code(city_name, token) -> str | None`

Steps:

1. Normalize `city_name`:
   - `strip()` whitespace, `lower()` for consistent keys.
2. Check `city_code_cache`:
   - If present, return cached code.
3. Check `COMMON_CITY_CODES`:
   - If present, store in cache and return.
4. Otherwise, call Amadeus **Locations API**:
   - URL: `BASE_URL + /v1/reference-data/locations`
   - Headers: `Authorization: Bearer <token>`
   - Two passes using different `subType` values:
     - `"CITY"`: find city‑level codes like `LON`, `MAA`.
     - `"AIRPORT,CITY"`: allow both airports and city codes, which helps if user typed an airport name (e.g. `Heathrow` → `LHR`).
   - For each pass:
     - Send GET with `params={"keyword": city_name, "subType": subtype}`.
     - If `data["data"]` is non‑empty, take the first `iataCode`, cache it, and return it.
5. If nothing is found after both attempts, return `None`.

In [13]:
city_code_cache = {}

def get_city_code(city_name: str, token: str) -> str | None:
    """
    Convert a city name (e.g. 'london') to an IATA location code (e.g. 'LON')
    using a small local dictionary first, then the Amadeus Locations API.
    """
    city_name = city_name.strip().lower()

    # Cache hit
    if city_name in city_code_cache: # First, check if we’ve already resolved this city name in a previous call.
        return city_code_cache[city_name] # If yes, immediately return the stored code.

    # Local mapping
    if city_name in COMMON_CITY_CODES: # If the city is in your hard‑coded mapping (e.g. "mumbai": "BOM"):
        code = COMMON_CITY_CODES[city_name] # Get that code.
        city_code_cache[city_name] = code # Save it in the cache.
        return code # Return it.

    # Amadeus reference-data/locations
    base_url = f"{BASE_URL}/v1/reference-data/locations" # If not in cache or local dictionary, you call the Amadeus Locations API.
    headers = {"Authorization": f"Bearer {token}"}

    # Try two different lookup modes with Amadeus:
    # 1) First pass: subType="CITY"
    #    - Amadeus will try to match the keyword as a city-level location.
    #    - Example:
    #        keyword = "london"  → might return IATA city code "LON"
    #        keyword = "chennai" → might return IATA city code "MAA"
    #    - If the user typed a pure city name, this usually works.
    
    # 2) Second pass: subType="AIRPORT,CITY"
    #    - Only used if the first pass finds nothing.
    #    - Now Amadeus is allowed to return both airports and cities.
    #    - This helps if the user typed an airport name instead of a city name.
    #    - Examples:
    #        keyword = "heathrow"        → can return airport code "LHR"
    #        keyword = "gatwick"         → can return airport code "LGW"
    #        keyword = "chennai airport" → might still resolve to "MAA" or a specific airport code
    
    # In practice:
    #   - If the user says "London"   → first pass ("CITY") is usually enough → "LON".
    #   - If the user says "Heathrow" → first pass likely returns nothing,
    #                                   second pass ("AIRPORT,CITY") can find "LHR".
    #   - Same idea for any other airport name or city+airport combination.
    for subtype in ["CITY", "AIRPORT,CITY"]:
        params = {"keyword": city_name, "subType": subtype}
        try:
            response = requests.get(base_url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if "data" in data and data["data"]:
                code = data["data"][0]["iataCode"]
                print(f"[INFO] Found {subtype} match for '{city_name}': {code}")
                city_code_cache[city_name] = code
                return code
        except Exception as e:
            print(f"[ERROR] Location lookup failed for {subtype}: {e}")

    return None

## 8. Live Ticket Price Lookup (`get_live_ticket_prices`)

### What it does

`get_live_ticket_prices(origin, destination, departure_date, return_date=None) -> str`:

- Validates dates.
- Authenticates with Amadeus.
- Resolves human city names to IATA codes.
- Calls the Amadeus **flight-offers search** endpoint.
- Picks the **top-ranked** offer (with `max=1`).
- Extracts airline name and price.
- Returns a **human-readable summary string**.

### Steps

1. **Local date validation**:
   - Parse `departure_date` as `%Y-%m-%d`.
   - If invalid, return an error message.
   - If in the past, return an error message.
   - If `return_date` is provided:
     - Parse it with the same format.
     - If invalid or before the departure date, return an error.

2. **Get OAuth token**:
   - Call `get_amadeus_token()`.
   - If `None`, return an authentication error message.

3. **Resolve cities to IATA codes**:
   - Call `get_city_code(origin, token)` and `get_city_code(destination, token)`.
   - If either fails, return a “could not find airport code” message.

4. **Build flight-offers search request**:
   - URL: `BASE_URL + /v2/shopping/flight-offers`.
   - Headers: `Authorization: Bearer <token>`.
   - Params:
     - `originLocationCode`, `destinationLocationCode`
     - `departureDate`
     - `adults=1`
     - `currencyCode="USD"`
     - `max=1` (ask for at most 1 offer)
     - Include `returnDate` if provided.

5. **Send request and handle errors**:
   - GET request with `requests.get(...)`.
   - `response.raise_for_status()` to handle HTTP errors.
   - On error, print details and return an error string.

6. **Parse offers**:
   - `offers = data.get("data", [])`.
   - If empty, return “No flights found...” message.
   - Otherwise, take `offer = offers[0]` (the top-ranked result).
   - Extract:
     - `price = offer["price"]["total"]`.
     - `airline_codes = offer.get("validatingAirlineCodes", [])`.
     - `airline_code = airline_codes[0]` or `"Unknown"`.
     - Use `get_airline_name` to map code to name.

7. **Format result**:
   - For round‑trip: include departure and return dates.
   - For one‑way: include only departure date.
   - Include airline name and total price.

In [14]:
def get_live_ticket_prices(origin: str,
                           destination: str,
                           departure_date: str,
                           return_date: str | None = None) -> str:
    """
    Get a human-readable summary of a flight offer from Amadeus (test environment).

    Args:
        origin: Origin city name (e.g. "london") or close variant.
        destination: Destination city name (e.g. "chennai").
        departure_date: Outbound date in 'YYYY-MM-DD' format.
        return_date: Optional return date in 'YYYY-MM-DD' format.

    Returns:
        A string describing the flight and price, or an error message.
    """

    # --- Local date validation before calling Amadeus ---
    today = date.today()

    try:
        dep = datetime.strptime(departure_date, "%Y-%m-%d").date()
    except ValueError:
        return f"Departure date '{departure_date}' is invalid. Use format YYYY-MM-DD."

    if dep < today:
        return f"Departure date {departure_date} is in the past. Please choose a future date."

    ret = None
    if return_date:
        try:
            ret = datetime.strptime(return_date, "%Y-%m-%d").date()
        except ValueError:
            return f"Return date '{return_date}' is invalid. Use format YYYY-MM-DD."

        if ret < dep:
            return f"Return date {return_date} is before departure date {departure_date}."

    # --- Get OAuth token ---
    token = get_amadeus_token() # Uses AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET.
    if not token: # Returns an access token string or None.
        return "Could not authenticate with Amadeus. Check your credentials and network connection."

    # --- Resolve city names to IATA codes ---
    origin_code = get_city_code(origin, token)
    destination_code = get_city_code(destination, token)

    if not origin_code:
        return f"Sorry, I couldn't find the airport code for the city '{origin}'."
    if not destination_code:
        return f"Sorry, I couldn't find the airport code for the city '{destination}'."

    # --- Build request to flight-offers search ---
    url = f"{BASE_URL}/v2/shopping/flight-offers"
    headers = {"Authorization": f"Bearer {token}"}

    params = {
        "originLocationCode": origin_code.upper(),
        "destinationLocationCode": destination_code.upper(),
        "departureDate": departure_date,
        "adults": 1,
        "currencyCode": "USD",
        "max": 1,
    }
    if return_date:
        params["returnDate"] = return_date

    # --- Call Amadeus API ---
    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError:
        print("[Amadeus HTTP error]")
        print("Status:", response.status_code)
        print("Body:", response.text)   # Inspect this to debug 400/500 errors
        return f"Error fetching flight data (HTTP {response.status_code})."
    except requests.exceptions.RequestException as e:
        return f"Error fetching flight data: {e}"

    # Ask Amadeus to return AT MOST 1 offer.
    # Internally, Amadeus finds many valid itineraries, ranks them using its own
    # relevance/quality logic (price, duration, stops, etc.), and then returns them
    # in that order. With max=1, we only receive the FIRST (top-ranked) offer.
    # So this is not a random flight; it is Amadeus' "best" option under our query,
    # though not necessarily the absolute cheapest in every case.

    # --- Parse offers ---
    offers = data.get("data", []) # Amadeus returns flight offers in data["data"] (a list).
    if not offers: # If it’s empty or missing, you tell the user no flights were found.
        return f"No flights found from {origin.capitalize()} to {destination.capitalize()} on {departure_date}."

    offer = offers[0] # You take the first offer (since max=1, there should be at most one anyway).
    price = offer["price"]["total"] # price["total"] is the total ticket price (string, e.g. "542.13").
    airline_codes = offer.get("validatingAirlineCodes", []) # validatingAirlineCodes gives the airline code(s) responsible for the ticket. E.g. ["EK"]
    airline_code = airline_codes[0] if airline_codes else "Unknown" # If there are multiple airlines, you only take the first one.

    try:
        airline_name = (
            get_airline_name(airline_code, token) # If you have a real airline code (not "Unknown"), you call get_airline_name: For example: "EK" → "Emirates".
            if airline_code != "Unknown"
            else "Unknown Airline"
        )
        if not airline_name:
            airline_name = airline_code
    except Exception:
        airline_name = airline_code

    # --- Format human-readable result ---
    if return_date:
        return (
            f"Round-trip flight from {origin.capitalize()} to {destination.capitalize()}:\n"
            f"- Departing: {departure_date}\n"
            f"- Returning: {return_date}\n"
            f"- Airline: {airline_name}\n"
            f"- Price: ${price}"
        )
    else:
        return (
            f"One-way flight from {origin.capitalize()} to {destination.capitalize()} on {departure_date}:\n"
            f"- Airline: {airline_name}\n"
            f"- Price: ${price}"
        )

## 9. Booking a Flight (`book_flight` + `booking_info`)

### What it does

`book_flight(origin, destination, departure_date, return_date=None, airline="Selected Airline", passenger_name="Guest")`:

- Simulates a booking by:
  - Generating a fake **PNR** (6‑character code of letters and digits).
  - Building a multi‑line confirmation message string.
  - Creating a `booking_info` dictionary with structured details.
- Returns **both**:
  - `confirmation` (string) – for the user and chat.
  - `booking_info` (dict) – for the boarding-pass image generator.

In [15]:
def book_flight(origin,
                destination,
                departure_date,
                return_date=None,
                airline="Selected Airline",
                passenger_name="Guest"):
    # Generate a dummy ticket reference (PNR)
    ticket_ref = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

    # Build confirmation message
    confirmation = (
        f"Booking confirmed for {passenger_name}!\n"
        f"From: {origin.capitalize()} → To: {destination.capitalize()}\n"
        f"Departure: {departure_date}"
    )

    if return_date:
        confirmation += f"\nReturn: {return_date}"

    confirmation += (
        f"\nAirline: {airline}\n"
        f"PNR: {ticket_ref}\n"
        f"Your ticket has been booked successfully. Safe travels!"
    )

    # Structured info for the boarding-pass image
    booking_info = {
        "passenger_name": passenger_name,
        "origin": origin.capitalize(),
        "destination": destination.capitalize(),
        "departure_date": departure_date,
        "return_date": return_date,
        "airline": airline,
        "pnr": ticket_ref,
    }

    # Return BOTH: text + structured data
    return confirmation, booking_info

## 10. Tool Declarations and Tools List
(`ticket_price_function_declaration`, `book_flight_function_declaration`, `TOOLS`)

These dictionaries describe your Python functions to the LLM in a JSON‑schema style. They are how you tell the model:

What tools exist.
When to use them.
Which arguments they require.

In [16]:
ticket_price_function_declaration = {
    "name":"get_live_ticket_prices",
    "description": "Get live flight ticket prices between two cities for a given date (round-trip or one-way).\
    The destination may be a city or country (e.g., 'China'). Call this function whenever a customer asks about ticket prices., such as 'How much is a ticket to Paris?'",
    "parameters":{
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "Name of the origin city. Example: 'Delhi'",
            },
             "destination": {
                "type": "string",
                "description": (
                    "Date of departure in YYYY-MM-DD format. "
                    "Never guess the year. If the user does not specify the year, "
                    "ask them to clarify before calling this function."
                ),
            },
            "departure_date": {
                "type": "string",
                "description": "Date of departure in YYYY-MM-DD format. Example: '2025-07-01'",
            },
            "return_date": {
                "type": "string",
                "description": "Optional return date for round-trip in YYYY-MM-DD format. Leave blank for one-way trips.",
            },
        },
        "required": ["origin", "destination", "departure_date"],
    }
}

In [17]:
book_flight_function_declaration = {
    "name": "book_flight",
    "description": "Book a flight for the user after showing the ticket details and confirming the booking. "
                   "Call this function when the user says things like 'yes', 'book it', or 'I want to book this flight'.",
    "parameters": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "Name of the origin city. Example: 'Chennai'",
            },
            "destination": {
                "type": "string",
                "description": "Name of the destination city. Example: 'London'",
            },
            "departure_date": {
                "type": "string",
                "description": "Date of departure in YYYY-MM-DD format. Example: '2025-07-01'",
            },
            "return_date": {
                "type": "string",
                "description": "Optional return date for round-trip in YYYY-MM-DD format. Leave blank for one-way trips.",
            },
            "airline": {
                "type": "string",
                "description": "Airline name or code that the user wants to book with. Example: 'Air India'",
            },
            "passenger_name": {
                "type": "string",
                "description": "Full name of the passenger for the booking. Example: 'Ravi Kumar'",
            }
        },
        "required": ["origin", "destination", "departure_date", "passenger_name"],
    }
}


## 11. System Prompt (`system_prompt`)
The `system_prompt` sets the global behavior and persona of the assistant:

In [18]:
system_prompt = (
    "You are a helpful, courteous AI assistant for an airline company called DoNotMakeMyTrip. "
    "When the user starts a new conversation (the first user message with no prior context), greet them with: "
    "\"Hi there, welcome to DoNotMakeMyTrip Airlines! How can I help you today?\" Do not repeat this greeting in follow-up messages. "
    "If a user asks about flight or ticket prices, always use the available pricing tools instead of guessing. "
    "Before calling a tool, ask any necessary follow-up questions to gather all required details, such as origin city, "
    "destination city, and travel dates. "
    "After calling a tool, summarize the result in natural language and then ask the user the next relevant question "
    "(for example, whether they want to proceed with a booking or modify their search). "
    "If you do not know the answer and no tool can help, respond politely that you are unable to help with the request. "
    "Answer concisely in one clear sentence."
)

In [20]:
# List of tools (functions) that the model can call during a chat.
TOOLS = [
    {"type": "function", "function": ticket_price_function_declaration},
    {"type": "function", "function": book_flight_function_declaration},
]


## 12. Boarding Pass Image Generation (`generate_boarding_pass_image`)
This function calls the image model to draw a boarding pass styled ticket, using the details from `booking_info`.

In [21]:
def generate_boarding_pass_image(
    passenger_name: str,
    origin: str,
    destination: str,
    departure_date: str,
    return_date: str | None,
    airline: str,
    pnr: str,
) -> Image.Image:
    """
    Generate a stylized boarding-pass-like ticket image using OpenAI's image model.
    Includes passenger name, from/to, dates, airline, PNR, and a fake QR code.
    """

    trip_desc = (
        f"Passenger: {passenger_name}, "
        f"From: {origin} to {destination}, "
        f"Departure: {departure_date}, "
    )
    if return_date:
        trip_desc += f"Return: {return_date}, "
    trip_desc += f"Airline: {airline}, PNR: {pnr}"

    prompt = (
        "Design a clean, realistic airline boarding pass for an airline called DoNotMakeMyTrip. "
        "Use a modern flat 2D graphic style on a light background. "
        "Include clearly readable fields for passenger name, origin, destination, departure date, "
        "return date (if provided), airline name, and PNR. "
        "Add a fake QR code on the right side of the card. "
        "Base all text details on the following booking information:\n\n"
        f"{trip_desc}\n\n"
        "Do not use any real airline logos. Show the details as printed text on the boarding pass."
    )

    response = openai.images.generate(
        model=MODEL_IMAGE,
        prompt=prompt,
        size="1024x1024",
        n=1,
        quality="hd",
        response_format="b64_json",  # important for dall-e-3
    )

    # Safely extract base64 image data
    image_b64 = response.data[0].b64_json
    if not image_b64:
        # Debug print if something goes wrong
        print("[Image Error] No base64 data returned by images.generate:")
        print(response)
        raise RuntimeError("Image generation did not return b64_json data.")

    image_bytes = base64.b64decode(image_b64)
    return Image.open(BytesIO(image_bytes))

## 13. Chat Logic with Tools (`chat`)
This is the core orchestration function for `model‑plus‑tools`:

his function is the central brain of your assistant. It decides:

- When to just answer with text.
- When to call a tool (price lookup or booking).
- How to feed the tool result back into the model.
- What to return to the UI (reply text, destination for image, booking info, updated history).
- It uses a two-step LLM call whenever a tool is involved.

### High-level flow
- Build a full messages list (system prompt + history + current user message).
- Call the OpenAI chat model once with tools enabled (tools=TOOLS, tool_choice="auto").
- Inspect the model’s reply:
    - If it includes a tool_calls field → the model is asking to run a tool.
    - If not → the model is just giving a normal text reply.
- If a tool is requested:
    - Run the Python function for that tool (e.g. get_live_ticket_prices or book_flight).
    - Add both:
        - The assistant’s tool-call message, and
        - The tool’s output (as a tool message) into the conversation.
    - Call the model a second time so it can read the tool result and produce a final, user‑friendly reply.
- Return:
    - The final reply text to show in the chat.
    - city_name (if a destination was involved) for destination images.
    - booking_info (if a booking was made) for the boarding‑pass image.
    - updated_history (conversation history including the new assistant message).


In [22]:
def chat(message: str, history: list[dict]) -> tuple[str, str | None, list[dict]]:
    """
    Main chat handler using OpenAI chat completions with tools.

    Args:
        message: Latest user message as a string.
        history: List of previous messages, each like {"role": "user"|"assistant", "content": "..."}.

    Returns:
        assistant_reply (str),
        city_name (str | None),
        booking_info (dict | None),
        updated_history (list[dict])
    """
    city_name: str | None = None
    booking_info: dict | None = None

    # Build the messages list for OpenAI:
    # 1) System prompt to control behavior
    # 2) Previous conversation history
    # 3) Current user message
    messages = [{"role": "system", "content": system_prompt}] + history + [
        {"role": "user", "content": message}
    ]

    # First call: let the model decide whether it needs to call a tool.
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",  # let the model decide when to call a tool
    )

    assistant_message = completion.choices[0].message
    # It tries to read assistant_message.tool_calls.
    # If assistant_message has a tool_calls attribute, tool_calls gets that value (a list of tool calls).
    # If it doesn’t (or it’s missing in this response), tool_calls is set to None.
    tool_calls = getattr(assistant_message, "tool_calls", None) 
    # if hasattr(assistant_message, "tool_calls"):
    #     tool_calls = assistant_message.tool_calls
    # else:
    #     tool_calls = None

    # Case 1: The model wants to call a tool (function)
    if tool_calls:
        # For simplicity, handle the first tool call only
        tool_call = tool_calls[0]
        function_name = tool_call.function.name
        # Arguments come as a JSON string; parse them to a dict
        function_args = json.loads(tool_call.function.arguments)

        # Execute the corresponding Python function
        if function_name == "get_live_ticket_prices":
            tool_result = get_live_ticket_prices(**function_args)
            dest = function_args.get("destination")
            if dest:
                city_name = dest.lower()
    
        elif function_name == "book_flight":
            # Now book_flight returns (confirmation, booking_info)
            confirmation, booking_info = book_flight(**function_args)
            tool_result = confirmation
            
        else:
            tool_result = f"Tool '{function_name}' is not implemented on the server."

        # Add the assistant message that contains the tool call
        # This recreates the assistant turn where the model requested a tool.
        # We explicitly store it in the messages list so the next model call
        # "remembers" that it asked to call a function.
        messages.append(
            {
                "role": "assistant",
                "content": None,
                "tool_calls": [tc.to_dict() for tc in tool_calls],
            }
        )

        # Add the tool result as a tool-role message so the model can see it
        messages.append(
            {
                "role": "tool", # This represents the *output* of the tool back into the conversation.
                "tool_call_id": tool_call.id, # tool_call_id links this output to the specific tool call the model made.
                # The model uses this to know which function call this result belongs to.
                "content": tool_result,
            }
        )

        # Second call: let the model generate a final, natural-language reply
        followup = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
        )

        final_text = followup.choices[0].message.content

        # Update the external history (we store only user/assistant, no system/tool)
        updated_history = history + [{"role": "assistant", "content": final_text}]
        return final_text, city_name, booking_info, updated_history


    # Case 2: No tool call; it's a plain text response
    else:
        reply_text = assistant_message.content
        updated_history = history + [{"role": "assistant", "content": reply_text}]
        return reply_text, city_name, None, updated_history

## 14. Gradio Bridge Function (`user_submit`)
This function glues the chat logic to the Gradio UI:

In [23]:
def user_submit(user_input, history):
    history = history or []
    history.append({"role": "user", "content": user_input})
    
    response_text, city_to_image, booking_info, updated_history = chat(user_input, history)

    audio_path = None
    # Speak the response
    try:
        audio_path = talk(response_text)
    except Exception as e:
        print("[Speech Error] Speech skipped due to quota limit.")

    # Decide what image to show
    # image = generate_company_logo()
    image = gr.update()
    if booking_info:
        # After booking: show boarding-pass image (replace destination photo)
        image = generate_boarding_pass_image(**booking_info)
    elif city_to_image:
        # After price lookup: show destination city image
        image = fetch_image(city_to_image)

    return "", updated_history, image, audio_path, updated_history


## 15. Gradio UI Setup and Launch
Finally, the Gradio Blocks interface:

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## DoNotMakeMyTrip Airline Assistant")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="Assistant", height=500, type="messages")
            msg = gr.Textbox(placeholder="Ask about flights...", show_label=False)
            send_btn = gr.Button("Send")

        with gr.Column(scale=2):
            initial_logo = generate_company_logo()
            image_output = gr.Image(label="Trip Visual", visible=True, value=initial_logo, height=500)
            audio_output = gr.Audio(label="Assistant voice", autoplay=True)

    state = gr.State([])
    
    send_btn.click(fn=user_submit, inputs=[msg, state], outputs=[msg, chatbot, image_output, audio_output, state],)
    msg.submit(fn=user_submit, inputs=[msg, state], outputs=[msg, chatbot, image_output, audio_output, state],)

demo.launch(inbrowser=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f5d6745fac8f8eedbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[INFO] Found CITY match for 'manchester': MAN
[INFO] Found CITY match for 'london': LON
